# LDA(Latent Dirichlet Allocation): 잠재 디리클레 할당

https://wikidocs.net/40710 참고

In [1]:
import pymysql

DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWD = 'autoset'
DB_NAME = 'python'

conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
    select * from yhnews
--    where newsDate between '20190101' and '20191231';

"""
curs.execute(sql)
table_data = curs.fetchall() 
conn.close()

In [2]:
import pandas as pd

df = pd.DataFrame(table_data,columns=['no','url','Date','Title','subtitle','Contents'])

# df [ 'Contents'   ]  <= Series타입,   df[ ['Contents'] ]  <== dataframe타입
sentences = df[ ['Contents'] ]   

# print로 type(text) 의 결과는 Dataframe타입이어야 함
print(type(sentences))
# <class 'pandas.core.frame.DataFrame'> 이 아니면 오류임

<class 'pandas.core.frame.DataFrame'>


라이브러리 추가 및 클래스 선언

In [10]:
# konlpy 설치 명령어
# !pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [3]:
from konlpy.tag import Okt
okt=Okt()

명사추출: nouns 명사추출. phrases 어절 추출

In [4]:
tokenized_doc = sentences.apply(lambda row: okt.nouns(row['Contents']), axis=1)


단어 길이가 1이하인 단어들은 제거 (길이가 짧은 단어 ex. 은/는/이/가/을/를 등 제거)


In [5]:
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if len(word) > 1])


분석에서 제거할 불필요한 단어들 기재

In [6]:
from nltk.corpus import stopwords

stop = ['연합뉴스','무단']
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if word not in (stop)])


In [7]:
# 저장된 단어들 확인하기 
print(tokenized_doc)

0     [광주, 송형일, 기자, 광주, 전남, 지역, 산업, 활동, 코로나, 충격, 전달,...
1     [서울, 코로나, 사태, 이후, 비관, 지표, 우리, 경제, 생산, 투자, 소비, ...
2     [서울, 코로나, 사태, 이후, 비관, 지표, 우리, 경제, 생산, 투자, 소비, ...
3     [정연주, 제작, 일러스트, 세종, 김연정, 차지연, 정수연, 기자, 생산, 소비,...
4     [여주, 최찬, 기자, 경기, 여주시, 추석, 명절, 이전, 농민, 수당, 지급, ...
5     [세종, 김연정, 차지연, 정수연, 기자, 신종, 코로나바이러스, 감염증, 코로나,...
6     [경향신문, 정권, 비호, 다수, 독주, 협치, 실종, 정치, 불신, 초래, 국민일...
7     [광주, 송형일, 기자, 광주, 전남, 지역, 산업, 활동, 코로나, 충격, 전달,...
8     [서울, 코로나, 사태, 이후, 비관, 지표, 우리, 경제, 생산, 투자, 소비, ...
9     [서울, 코로나, 사태, 이후, 비관, 지표, 우리, 경제, 생산, 투자, 소비, ...
10    [정연주, 제작, 일러스트, 세종, 김연정, 차지연, 정수연, 기자, 생산, 소비,...
11    [여주, 최찬, 기자, 경기, 여주시, 추석, 명절, 이전, 농민, 수당, 지급, ...
12    [세종, 김연정, 차지연, 정수연, 기자, 신종, 코로나바이러스, 감염증, 코로나,...
13    [경향신문, 정권, 비호, 다수, 독주, 협치, 실종, 정치, 불신, 초래, 국민일...
14    [서울, 미숙, 최재서, 기자, 신종, 코로나바이러스, 감염증, 코로나, 여파, 우...
dtype: object


TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치

https://wikidocs.net/31698 참고

TF-IDF의 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용함.
이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행

# 사이킷런의 잠재 디리클레 할당(LDA) 실습

https://wikidocs.net/40710

In [8]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []

for i in range(len(sentences)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

sentences['Contents'] = detokenized_doc

print(sentences['Contents'][:5])

0    광주 송형일 기자 광주 전남 지역 산업 활동 코로나 충격 전달 다소 개선 호남 통계...
1    서울 코로나 사태 이후 비관 지표 우리 경제 생산 투자 소비 동시 통계청 발표 산업...
2    서울 코로나 사태 이후 비관 지표 우리 경제 생산 투자 소비 동시 통계청 발표 산업...
3    정연주 제작 일러스트 세종 김연정 차지연 정수연 기자 생산 소비 투자 산업 활동 지...
4    여주 최찬 기자 경기 여주시 추석 명절 이전 농민 수당 지급 여주시 제공 재판매 금...
Name: Contents, dtype: object


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20588\3324690400.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences['Contents'] = detokenized_doc


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features= 1000)    # 상위 1,000개의 단어를 보존 

# TfidfVectorizer 할 X_train 데이터
X = vectorizer.fit_transform(sentences['Contents'])

from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델훈련
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_model.fit_transform(X)

# vocabulary를 활용하여 각 문장이 갖고 있는 토큰의 count를 기반으로 문장을 vectorization 처리
terms = vectorizer.get_feature_names() 

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

#  토픽별 단어 분포
get_topics(lda_model.components_,terms)



Topic 1: [('차질', 1.23), ('승인', 1.14), ('변화', 1.14), ('파이', 1.11), ('동월', 1.11), ('타결', 1.07), ('주신문', 1.07), ('요인', 1.06), ('이야기', 1.06), ('이익', 1.06)]
Topic 2: [('상정', 1.1), ('시중', 1.1), ('입자', 1.1), ('신청', 1.1), ('정치', 1.07), ('자연', 1.05), ('가전', 1.04), ('전자신문', 1.04), ('통제', 1.04), ('커스텀슈즈', 1.04)]
Topic 3: [('총동원', 1.06), ('양산', 1.06), ('준비', 1.05), ('재고', 1.05), ('매머드', 1.05), ('가공', 1.04), ('검토', 1.04), ('실종', 1.03), ('반면', 1.03), ('유통', 1.03)]
Topic 4: [('차기', 1.11), ('집중', 1.08), ('마트', 1.07), ('위축', 1.04), ('전대', 1.04), ('입지', 1.03), ('실행', 1.03), ('국내총생산', 1.03), ('모멘텀', 1.02), ('명절', 1.02)]
Topic 5: [('혼돈', 1.13), ('공업', 1.07), ('재원', 1.07), ('하자', 1.05), ('여파', 1.04), ('성도', 1.03), ('원금', 1.03), ('수첩', 1.02), ('독주', 1.02), ('변화', 1.02)]
Topic 6: [('주력', 1.09), ('매출', 1.06), ('금강', 1.05), ('행사', 1.04), ('전남', 1.04), ('위축', 1.04), ('공공기관', 1.04), ('온라인', 1.03), ('앞서', 1.03), ('재개', 1.03)]
Topic 7: [('추경', 1.11), ('일보', 1.1), ('금남로', 1.07), ('바이러스', 1.07), ('대형', 1.06), ('오라',

C:\Users\for\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# gensim을 통해서 LDA를 수행하고, 시각화

https://wikidocs.net/30708

In [10]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [11]:
import gensim

NUM_TOPICS = 6 # n개의 토픽, k=n

# passes는 알고리즘의 동작 횟수
# 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 설정함
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)   # TOPIC별 단어를 4개만 보여주기, 숫자를 늘리면 더 많은 단어들이 보여짐
for topic in topics:
    print(topic)


(0, '0.023*"생산" + 0.023*"경제" + 0.023*"성장" + 0.019*"투자"')
(1, '0.048*"일보" + 0.016*"공항" + 0.015*"신문" + 0.013*"매일"')
(2, '0.025*"이익" + 0.025*"매출" + 0.025*"영업" + 0.023*"작년"')
(3, '0.038*"대비" + 0.030*"증가" + 0.030*"동월" + 0.028*"감소"')
(4, '0.046*"생산" + 0.046*"증가" + 0.030*"코로나" + 0.024*"산업"')
(5, '0.001*"생산" + 0.001*"증가" + 0.001*"일보" + 0.001*"코로나"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 N개의 토픽은 0부터 N-1까지의 번호가 할당되어져 있음

위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미함
예를 들어 0번째 문서의 토픽 비율에서 (0, '0.023*"경제..."'')은 0번 토픽에서 경제가 2.3%의 분포도를 가지는 것을 의미함

In [15]:
# !pip install pyLDAvis

In [14]:

import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

# https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/
# 시각화는 1- 부터 시작함
pyLDAvis.display(vis)

C:\Users\for\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


λ라는 값은 [0, 1] 사이에서 조절가능함
λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며,
λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미임